# Schematics 4.0 — FaceStats
Updated diagrams reflecting the current v4 repo layout and data flow.

## Tools

```mermaid
flowchart LR
    subgraph PRE[Preprocessing]
        MP[Mediapipe FaceMesh]
        CV2[OpenCV]
        PIL[Pillow]
        TQDM[tqdm]
    end
    subgraph EMB[Embeddings]
        HF[Transformers (CLIP)]
        TORCH[PyTorch]
    end
    subgraph ATTR[Attributes]
        SK[sklearn classifiers (joblib)]
    end
    subgraph DATA[Data]
        PL[Polars (notebook)]
        NP[NumPy]
    end
    subgraph VIZ[Visualization]
        MPL[Matplotlib/Seaborn]
    end

    PRE --> EMB
    EMB --> ATTR
    EMB --> DATA
    ATTR --> DATA
    DATA --> VIZ
```


## Diagram 1 — FaceStats v4 High Level Pipeline

```mermaid
flowchart LR
    A["📁 Raw Images
 data/raw/"] --> B["🧹 Preprocess
 src/pipeline/preprocess.py -> data/processed/preproc/"]
    B --> C["🧠 CLIP Embeddings
 notebooks/02_embeddings.ipynb -> data/processed/embeddings/embeddings_clip.parquet"]
    C --> D["👥 Attributes
 notebooks/03_attributes.ipynb -> data/processed/metadata/attributes.parquet"]
    C --> E["💚 Attractiveness Train
 notebooks/04_attractiveness_model.ipynb -> models/attractiveness_regressor.pt"]
    C --> F["💚 Attractiveness Inference
 notebooks/05_attractiveness_inference.ipynb -> data/processed/metadata/attractiveness_scores.parquet"]
    D --> G["📊 Clean/Flags/Meta/Clusters
 data/processed/metadata/*.parquet"]
    F --> G
    G --> H["🎨 Composites/Analysis
 notebooks/05_composites.ipynb, 04_visualize_attributes.ipynb"]
```


## Diagram 2 — Preprocessing Flow

```mermaid
flowchart LR
    A["📁 data/raw/"] --> B["🔍 Load
Pillow"]
    B --> C["🧽 RGB + Normalize"]
    C --> D["📏 Resize 512x512"]
    D --> E["💾 Save JPG
 data/processed/preproc/"]
```


## Diagram 3 — Embedding Extraction

```mermaid
flowchart LR
    A["🖼️ Preprocessed
 data/processed/preproc/"] --> B["📥 Tokenize
 CLIPProcessor"]
    B --> C["🧠 Forward
 CLIP/ViT (torch + HF)"]
    C --> D["📏 L2 Normalize
 512-D"]
    D --> E["💾 Parquet
 data/processed/embeddings_clip.parquet"]
```


## Diagram 4 — Attributes

```mermaid
flowchart LR
    A["🧠 Embeddings"] --> B["👤 Gender/Ethnicity (sklearn)
 src/attributes/face_attributes.py"]
    B --> C["📄 attributes.parquet"]
    C --> D["✅ attributes_clean.parquet"]
    C --> E["🚩 attributes_flags.parquet"]
    C --> F["🌍 attributes_with_clusters.parquet"]
    C --> G["🔗 attributes_with_meta.parquet / attributes_with_manual.parquet / attributes_with_predictions.parquet"]
```


## Diagram 5 — Attractiveness

```mermaid
flowchart LR
    A["🔢 Embeddings
 data/processed/embeddings/embeddings_clip.parquet"] --> B["🧠 Train MLP
 notebooks/04_attractiveness_model.ipynb"]
    B --> C["💾 Model
 models/attractiveness_regressor.pt"]
    A --> D["🏃 Inference
 notebooks/05_attractiveness_inference.ipynb"]
    C --> D
    D --> E["📄 Scores
 data/processed/metadata/attractiveness_scores.parquet"]
```


## Diagram 6 — Repo Map (Current)

```text
FaceStats/
├── LICENSE
├── README.md
├── requirements.txt
├── schematics.ipynb
├── schematics_4_0.ipynb (this file)
├── tools_summary.md, repo_map.md
├── del.ipynb, x.ipynb
├── data/
│   ├── raw/
│   │   └── fairface/
│   ├── processed/
│   │   ├── preproc/ (resized JPGs)
│   │   ├── embeddings/
│   │   │   └── embeddings_clip.parquet
│   │   ├── metadata/
│   │   │   ├── attributes.parquet, attributes_clean.parquet, attributes_final.parquet
│   │   │   ├── attributes_flags.parquet, attributes_with_clusters.parquet
│   │   │   ├── attributes_with_meta.parquet, attributes_with_manual.parquet, attributes_with_predictions.parquet
│   │   │   ├── attractiveness_scores.parquet, attractiveness_with_attributes.parquet
│   │   │   ├── fairface_label_structure.parquet, feature_index.json
│   │   │   └── labels_template.csv, manual_labels.csv
│   │   ├── composites/ (e.g., composite_v4_example.jpg)
│   │   └── attractiveness_scores.npy
│   ├── interim/
│   │   ├── checkpoints/
│   │   └── preprocessed/
│   ├── embeddings/
│   ├── models/
│   └── attributes/ (legacy)
├── models/
│   ├── attractiveness_regressor.pt
│   └── gender_clf.pkl
├── models_insightface/
├── notebooks/
│   ├── 01_preprocess.ipynb ... 06_composites.ipynb, data/
├── src/
│   ├── pipeline/, embeddings/, attributes/, models/, composite/, data_utils/, learning/, metadata/, visualization/
└── config/
```
